In [1]:
source bioinf_intro_config.sh
rm -rf $IGV_DIR $STAR_OUT
mkdir -p $TRIMMED $STAR_OUT $IGV_DIR

# Run with shorter intron limit

IGV needs indices for the BAM files.  The index allows it to quickly load reads from different parts of the genome.

In [2]:
for FASTQ in $RAW_FASTQS/21_2019_P_M1_S21_L00[1-2]_R1_001.fastq.gz
    do
        FASTQ_BASE="$(basename ${FASTQ} '_001.fastq.gz')"
        echo "---------------- TRIMMING: $FASTQ_BASE ----------------"
        fastq-mcf \
            $MYINFO/neb_e7600_adapters.fasta \
            $RAW_FASTQS/${FASTQ_BASE}_001.fastq.gz \
            -q 20 -x 0.5 \
            -o $TRIMMED/${FASTQ_BASE}_001.trim.fastq.gz
        
        echo "---------------- MAPPING: $FASTQ_BASE ----------------"
        STAR \
            --runMode alignReads \
            --twopassMode None \
            --genomeDir $GENOME_DIR \
            --readFilesIn $TRIMMED/${FASTQ_BASE}_001.trim.fastq.gz \
            --readFilesCommand gunzip -c \
            --outFileNamePrefix ${STAR_OUT}/${FASTQ_BASE}_short_introns_ \
            --quantMode GeneCounts \
            --outSAMtype BAM SortedByCoordinate \
            --alignIntronMax 5000 \
            --outSJfilterIntronMaxVsReadN 500 1000 2000  \
            --runThreadN 2
            
        echo "---------------- INDEXING BAM: $FASTQ_BASE ----------------"
        samtools index ${STAR_OUT}/${FASTQ_BASE}_short_introns_Aligned.sortedByCoord.out.bam
    done

---------------- TRIMMING: 21_2019_P_M1_S21_L001_R1 ----------------
Command Line: /home/jovyan/work/scratch/bioinf_intro/myinfo/neb_e7600_adapters.fasta /data/hts_2019_data/hts2019_pilot_rawdata/21_2019_P_M1_S21_L001_R1_001.fastq.gz -q 20 -x 0.5 -o /home/jovyan/work/scratch/bioinf_intro/trimmed_fastqs/21_2019_P_M1_S21_L001_R1_001.trim.fastq.gz
Scale used: 2.2
Phred: 33
Threshold used: 751 out of 300000
Adapter Adapter (AGATCGGAAGAGCACACGTCTGAACTCCAGTCA): counted 2504 at the 'end' of '/data/hts_2019_data/hts2019_pilot_rawdata/21_2019_P_M1_S21_L001_R1_001.fastq.gz', clip set to 6
Files: 1
Total reads: 2479050
Too short after clip: 1519
Clipped 'end' reads: Count: 46013, Mean: 15.47, Sd: 8.24
Trimmed 299018 reads by an average of 1.71 bases on quality < 20
---------------- MAPPING: 21_2019_P_M1_S21_L001_R1 ----------------
Jun 26 16:41:58 ..... started STAR run
Jun 26 16:41:58 ..... loading genome
Jun 26 16:42:02 ..... started mapping
Jun 26 16:42:35 ..... started sorting BAM
Jun 26 16:4

In [3]:
ls -ltr ${STAR_OUT}

total 204980
-rw-r--r-- 1 jovyan users    166944 Jun 26 16:42 21_2019_P_M1_S21_L001_R1_short_introns_ReadsPerGene.out.tab
-rw-r--r-- 1 jovyan users 104774284 Jun 26 16:42 21_2019_P_M1_S21_L001_R1_short_introns_Aligned.sortedByCoord.out.bam
-rw-r--r-- 1 jovyan users    918773 Jun 26 16:42 21_2019_P_M1_S21_L001_R1_short_introns_SJ.out.tab
-rw-r--r-- 1 jovyan users       246 Jun 26 16:42 21_2019_P_M1_S21_L001_R1_short_introns_Log.progress.out
-rw-r--r-- 1 jovyan users     18641 Jun 26 16:42 21_2019_P_M1_S21_L001_R1_short_introns_Log.out
-rw-r--r-- 1 jovyan users      1849 Jun 26 16:42 21_2019_P_M1_S21_L001_R1_short_introns_Log.final.out
-rw-r--r-- 1 jovyan users     32536 Jun 26 16:42 21_2019_P_M1_S21_L001_R1_short_introns_Aligned.sortedByCoord.out.bam.bai
-rw-r--r-- 1 jovyan users    166808 Jun 26 16:43 21_2019_P_M1_S21_L002_R1_short_introns_ReadsPerGene.out.tab
-rw-r--r-- 1 jovyan users 102828697 Jun 26 16:43 21_2019_P_M1_S21_L002_R1_short_introns_Aligned.sortedByCoord.out.bam
-rw-r--r-

#### Link Directory

In [5]:
ln -s ${STAR_OUT}/*.bam* $GENOME_DIR/$GTF $GENOME_DIR/$FA $IGV_DIR

#### Tarring

In [6]:
tar --dereference \
    --create \
    --gzip \
    --verbose \
    --file $CUROUT/stuff_for_igv.tgz \
    --directory $CUROUT \
    $(basename $IGV_DIR)

igv/
igv/Cryptococcus_neoformans_var_grubii_h99.CNA3.39.gtf
igv/21_2019_P_M1_S21_L002_R1_short_introns_Aligned.sortedByCoord.out.bam
igv/21_2019_P_M1_S21_L001_R1_short_introns_Aligned.sortedByCoord.out.bam.bai
igv/Cryptococcus_neoformans_var_grubii_h99.CNA3.dna.toplevel.fa
igv/21_2019_P_M1_S21_L002_R1_short_introns_Aligned.sortedByCoord.out.bam.bai
igv/21_2019_P_M1_S21_L001_R1_short_introns_Aligned.sortedByCoord.out.bam
